In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import SimpleITK as sitk
import numpy as np
%matplotlib inline

In [2]:
def swc_to_dataframe( file_name ) :
    col_names = ['id', 'type', 'x', 'y', 'z', 'radius', 'parent', 'seg_id', 'level', 'mode', 'timestamp', 'TFresindex']
    df = pd.read_csv( file_name, sep=r"\s+", comment = "#", names = col_names, index_col = 'id')
    return df

In [3]:
def filter_swc( df, type ) :
    dd = df[df['type']==type]
    return dd

In [4]:
model_directory = "/allen/programs/celltypes/production/0378/informatics/model/P56"
image_directory = os.path.join( model_directory, "atlasVolume")

model_directory = "/home/lydian/informatics_lydian/Morphology/WholeToSliceView"
swc_directory = os.path.join( model_directory, "aibs", "73_cells", "sorted")
output_directory = os.path.join( model_directory, "aibs", "73_cells", "sorted_thumbnail")

In [5]:
#
# component types
#
types = {}
types['soma'] = {'type_id': 1, 'color': 'black'}
types['axon'] = {'type_id': 2, 'color': 'cyan'}
types['basal dendrite'] = {'type_id': 3, 'color': 'orangered'}
types['apical dendrite'] = {'type_id': 4, 'color': 'maroon'}

#
# views for swc
#
views = {}
views['coronal'] = {'order': 0, 'x': 'z', 'y': 'y'} #x
views['sagittal'] = {'order': 1, 'x': 'x', 'y' : 'y'} #y
views['horizontal'] = {'order': 2, 'x': 'x', 'y' : 'z'} #z

#
# image slices
#
slices = {}
slices['coronal'] = {'swc': 'x', 'image' : 'x', 'transpose' : True }
slices['sagittal'] = {'swc': 'z', 'image' : 'z', 'transpose' : False }
slices['horizontal'] = {'swc' : 'y', 'image' : 'y', 'transpose' : False }

#
# scale from image pixel index to SWC coordinate
#
scales = {}
scales['z'] = 10
scales['x'] = 10
scales['y'] = 10

shifts = {}
shifts['z'] = 0
shifts['x'] = 0
shifts['y'] = 0

In [6]:
#
# Create morphology thumbnail for each SWC file in the swc directory
#
#

image_file = os.path.join(image_directory, "average_template_10.nrrd" )
image = sitk.ReadImage( image_file )
print(image.GetSize())

for swc_file in glob.glob(os.path.join(swc_directory, "*.swc")):
    
    bb = os.path.basename( swc_file )
    
    # find the associated brain file
    bname = bb.split('_')[0]
    
    # set the output thumbnail png file name
    cname = bb.split('.')[0]
    thumnail_file = os.path.join( output_directory, "%s.png" % cname )
    
    print((cname,bname))
    
    #
    # read swc components into dataframe
    #
    df = swc_to_dataframe( swc_file )


    #
    # scale swc coordinates to match image volume
    #
    for sk, sv in shifts.items() :
        df[sk] = df[sk] + sv

    for sk, sv in scales.items() :
        df[sk] = df[sk] / sv
        

    #
    # get background image from soma location
    #

    filtered = filter_swc(df,types['soma']['type_id'])
    
    if len(filtered) == 0 :
        print("no soma found")
        continue
        
    soma = filtered.iloc[0]
 
    #
    #  plot swc components onto of image at soma location
    #
    plt.rcParams["figure.figsize"] = (30,8)

    fig, axs = plt.subplots(1,3)
    fig.suptitle(os.path.basename(swc_file))

    for vk, vv in views.items() :

        idx = int(round(soma[slices[vk]['swc']],0))

        if slices[vk]['image'] == 'x' :
            bkg = sitk.GetArrayViewFromImage( image )[:,:,idx]
        elif slices[vk]['image'] == 'y' :
            bkg = sitk.GetArrayViewFromImage( image )[:,idx,:]
        elif slices[vk]['image'] == 'z' :
            bkg = sitk.GetArrayViewFromImage( image )[idx,:,:]

        if slices[vk]['transpose'] :
            bkg = bkg.transpose()

        axs[vv['order']].imshow( bkg, cmap='gray')

        for tk,tv in types.items() :

            filtered = filter_swc(df, tv['type_id'])

            x = filtered[vv['x']].values
            y = filtered[vv['y']].values

            axs[vv['order']].scatter( x, y, s=0.01, c=tv['color'])

        axs[vv['order']].set_aspect('equal')

    
    plt.savefig(thumnail_file, dpi = 150)
    plt.close("all")
    

(1320, 800, 1140)
('18864_5637-X5637-Y31619_reg', '18864')
no soma found
('18869_6854-X10735-Y2978_reg', '18869')
('18453_4283-X7739-Y13735_reg', '18453')
('18453_6756-X7677-Y5336_reg', '18453')
no soma found
('17109_6601-X4384-Y7436_reg', '17109')
('18867_6985-X12525-Y4095_reg', '18867')
('18453_5667-X25695-Y3103_reg', '18453')
('18868_6757-X10353-Y4324_reg', '18868')
('18864_6321-X4387-Y7047_reg', '18864')
no soma found
('18453_7304-X11719-Y3749_reg', '18453')
no soma found
('18864_6139-X4843-Y27122_reg', '18864')
('17300_6010-X23413-Y13446_reg', '17300')
('18453_6053-X8032-Y5933_reg', '18453')
('182725_7341-X4593-Y22594_reg', '182725')
('18867_7584-X10560-Y5816_reg', '18867')
('18867_7390-X9887-Y4961_reg', '18867')
('18453_7010-X8769-Y4509_reg', '18453')
no soma found
('191812_5154-X9610-Y25921_reg', '191812')
('18453_6181-X10238-Y3690_reg', '18453')
no soma found
('18453_6479-X9010-Y4330_reg', '18453')
no soma found
('17543_5334-X5066-Y20381_reg', '17543')
('18453_7223-X30215-Y5120